In [1]:
import os
import json
from groq import Groq

In [2]:
#open dataset and store it as a JSON object
with open('../GSM-IC/GSM-IC_2step.json', 'r') as file:
    questions = json.load(file)

In [3]:
#example of an entry in the dataset
#todo: what's the point of all the keys under 'n_steps'?
questions[0]

{'original_question': 'Steve is 5\'6".  He grows 6 inches.  How tall is he in inches?',
 'answer': '72',
 'new_question': 'Steve is 5\'6". He grows 6 inches. The height of Emma is 8 feet. How tall is Steve in inches?',
 'n_steps': 2,
 'role': 'Emma',
 'number': '8',
 'sentence_template': 'The height of {role} is {number} feet.',
 'role_label': 'nonoverlapped',
 'number_label': 'in_range',
 'sentence_label': 'in_topic'}

In [4]:
#initialize the Groq client. This allows us to query Llama3 (or another model) from my Groq account
#todo: store the api_key in a secure way!
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY")
    api_key='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'
)

In [5]:
#todo: 1. answer_prompt is the current way of forcing the model to output its answer in predictable format. Find a better way to extract the numerical answer from the model's output. (maybe: require JSON output)

#todo: 2. clean up the answer. Ex. remove $, %, inches, etc. 

def single_query(query):
    """
    query the model with a single question
    input: what you would type in the chat box to ask the model a question
    output: the model's output in response to your query 
    """

    
    answer_prompt='The last sentence of your response should be in the form "The answer is [your answer]".'
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query + answer_prompt,
        }
    ],
    model="llama3-8b-8192",
    )
    response = chat_completion.choices[0].message.content
    return response 


In [9]:
#todo: shouldn't need to separate the two queries like this
def multiple_queries(question_type='original_question', num_questions=5):
    """
    query the model sequentially with multiple questions from the dataset 
    Args: question_type: specify whether to query the model with the original question or the new question 
        num_questions: the number of questions (mainly for testing a few questions at a time)
    
    """
    for idx, question in enumerate(questions):
        if idx <num_questions:
            query = question[question_type]
            response = single_query(query)
            print(response)
            print('--------------------------------------------------')

In [10]:
#query with relevant information only
multiple_queries(question_type='original_question', num_questions=5)


Let's convert Steve's height from feet and inches to inches. Since 1 foot is equal to 12 inches, Steve is initially 66 inches tall (5'6" = 5 x 12 + 6).

Then, he grows 6 inches, so we add that to his initial height:

66 + 6 = 72

The answer is 72.
--------------------------------------------------
Let's break it down:

Jewel bought 10 magazines at $3 each, so she spent:
10 x $3 = $30

She will sell the magazines at $3.50 each, so she will gain:
10 x $3.50 = $35

To find out how much Jewel will gain, we subtract the cost from the revenue:
$35 - $30 = $5

The answer is $5.
--------------------------------------------------
Edmund uses 18 inches of wrapper per gift box. In 3 days, he has 90 x 3 inches of wrapper, which is 270 inches of wrapper. How many 18's fit into 270? 270 / 18 = 15. The answer is 15.
--------------------------------------------------
Since the bar lasts for 2 months, Maria will need 6 bars for the entire year (12 months ÷ 2 months per bar = 6 bars). If each bar costs 

In [11]:
#query with irrelevant information included
#todo: merge with querying 'original_question' to more easily compare differences in model output
multiple_queries(question_type='new_question', num_questions=5)

Let's convert Steve's original height to inches. There are 12 inches in a foot, so 5'6" is equal to:

5 feet x 12 inches/foot = 60 inches
+ 6 inches = 66 inches

Steve grows 6 more inches, so his new height is:

66 inches + 6 inches = 72 inches

Emma is 8 feet tall, which is equal to:

8 feet x 12 inches/foot = 96 inches

So, Steve's height in inches is 72 inches. The answer is 72.
--------------------------------------------------
Let's first calculate the cost of the magazines that Jewel bought: 10 magazines x $3 each = $30.

Jewel will sell the magazines for $3.50 each, so she will earn:

10 magazines x $3.50 each = $35

To find out how much Jewel will gain, we subtract the cost from the selling price:

$35 (selling price) - $30 (cost) = $5

The answer is $5.
--------------------------------------------------
Each day, Edmund has 90 inches of gift wrapper. Every 3 days, he has 90 * 3 = 270 inches. Each gift box uses 18 inches of gift wrapper. 270 ÷ 18 = 15, so he can wrap 15 gift bo